# 基本流程

决策树是一类常见的机器学习方法。决策树是基于树结构来进行决策的。

.例如，我们要对“这是好瓜吗?”这样的问题进行决策时，通常会进行一系列的判断或“子决策”：我们先看“它是什么颜色?”，如果是“青绿色”，则我们再看“它的根蒂是什么形态?”，如果是“蜷缩”，我们再判断“它敲起来是什么声音?”，最后，我们得出最终决策：这是个好瓜.

一般的，一颗决策树包含一个根节点、若干个内部节点和若干个叶节点；叶节点对应于决策结果，其他每个结点则对应于一个属性的测试，每个节点包含的样本集合根据属性测试结果划分到子结点中；根节点包含整个样本集。这种流程叫“分而治之”（divide-and-conquer)

决策树是一个递归的过程。有三种情况导致递归返回:

1. 当前节点包含的样本所有属性属于同一类别，无需划分；

2. 当前属性集为空，所有样本在所有属性上的取值相同，无法划分；

    解决：把当前节点标记为叶结点，将其类别设定为该节点所含样本最多的类别；（当前节点的后验分布）

3. 当前节点包含的样本集合为空，不能划分。
    
    解决：把当前节点标记为叶节点，将其类别设定为父结点所含样本最多的类别。（当前节点的先验分布）

# 划分选择

## 信息增益
   
### 熵 entropy

    熵是随机变量不确定的度量。

    假设X是一个有限状态的离散型随机变量，其概率分布为： 
$$P(X=x_{i}) = p_{i}, i=1,2,...,n$$

    那么随机变量X的熵定义为： 
    
$$H(X) = - \sum_{i=1}^{n} p_{i} log(p_{i})$$

    熵越大，则随机变量的不确定性就越大** 或者说 **熵越大，随机变量的纯度（purity）就越低


    当随机变量只有0，1两个取值时:

    假设$P(X=1) = p$, 则随机变量X的熵定义为 
$$H(X) = -plog(p) -(1-p)log(1-p)$$
    
    求导可知，当p=0.5时，熵取值最大，随机变量不确定性最大。


#### 条件熵

    随机变量X给定的条件下，随机变量Y的条件熵$H(Y|X)$定义为：

$$H(Y|X) = \sum_{i=1}^{n} p_{i} H(Y|X=x_{i})$$
    
    其中，$p_{i} = P(X = x_{i})$



### 信息增益

根据信息增益准则进行特征选择的方法是：对训练数据集D，计算其每个特征的信息增益，并比它们的大小，从而选择信息增益最大的特征.

以信息增益作为特征选择准则，会存在偏向于选择取值较多的特征的问题。可以采用信息增益比对这一问题进行校正。

根据信息增益准则进行特征选择的方法是：对训练数据集D，计算其每个特征的信息增益，并比它们的大小，从而选择信息增益最大的特征。

**假设训练数据集为D，样本容量为|D|,有$k$个类别$C_{k}$,$|C_{k}|$为类别$C_{k}$的样本个数。某一特征A有n个不同的取值$a_{1},...,a_{n}$。根据特征A的取值可将数据集D划分为n个子集$D_{1}，...,D_{n}$,$|D_{i}|$为$D_{i}$的样本个数。并记子集$D_{i}$中属于类$C_{k}$的样本的集合为$D_{ik}$,$|D_{ik}|$为$D_{ik}$的样本个数。**


假设样本第k类样本所占的比例为$p_{k}(k=1,2,...|Y|)$,那么信息熵则为：

$$H(D) = - \sum_{k=1}^{|Y|} p_{k} \log_{2}(p_{k})$$

### 信息熵增


得知特征X的信息而使得类Y的信息的不确定性减少的程度。

特征A对训练数据集D的信息增益$g(D,A)$定义为集合D的经验熵$H(D)$与特征A给定条件下D的经验条件熵$H(D|A)$之差，即

$$g(D, A) = H(D) - H(D|A)$$

### 增益率

简单使用信息增益作为准则有一个较大的问题，就是它更偏好取值较多的属性。例如，当相貌这个属性有帅和不帅两个取值，而身家这个属性有贫穷，小康，中产，富裕四个取值，那么信息增益会更偏好身家这个属性。为了减少这个英雄，C4.5使用增益率来选择最优划分属性。

特征A对训练数据集D的**信息增益率**定义为其信息增益与训练集D关于特征A的值的熵之比，即

$$gR(D|A) = \frac{g(D,A)}{H_{A}(D)}$$

其中，$$H_{A}(D) = -\sum_{i=1}^{n} \frac{|Di|}{|D|}log(\frac{|Di|}{|D|})$$

**C4.5决策树算法**[Quinlan, 1993]就不直接使用信息增益，而是使用“增益率”(gain ratio/gR)来选择最优划分属性.

In [19]:
# 对y的各种可能的取值出现的个数进行计数.。其他函数利用该函数来计算数据集和的混杂程度
def uniquecounts(rows):
    results = {}
    for row in rows:
        #计数结果在最后一列
        r = row[len(row)-1]
        if r not in results:results[r] = 0
        results[r]+=1
    return results # 返回一个字典

# 熵

def entropy(rows):
    from math import log
    log2 = lambda x:log(x)/log(2)
    results = uniquecounts(rows)
    #开始计算熵的值
    ent = 0.0
    for r in results.keys():
        p = float(results[r])/len(rows)
        ent = ent - p*log2(p)
    return ent

#### 1.6 ID3 算法和C4.5算法

ID3算法的核心是在决策树的各个结点上应用信息增益准则进行特征选择。具体做法是：


从根节点开始，对结点计算所有可能特征的信息增益，选择信息增益最大的特征作为结点的特征，并由该特征的不同取值构建子节点；
对子节点递归地调用以上方法，构建决策树；
直到所有特征的信息增益均很小或者没有特征可选时为止。

C4.5算法与ID3算法的区别主要在于它在生产决策树的过程中，使用信息增益比来进行特征选择。

### 基尼指数（GINI INDEX）

**CART决策树**就是使用基尼指数来选择划分属性。


数据集D的基尼值：$$Gini(D) = \sum_{i=1}^{n}\sum_{i'\neq i} p_{i}p_{i'} = 1 - \sum_{i=1}^{n} p_{k}^{2}$$

Gini(D)反映的是从数据集D中随机抽取两个样本，其类别标志不一致的概率。所以Gini（D）越小，D的纯度就越高。

属性a的基尼指数则定义为：

$ Gini Index(D,a) = \sum_{v=1}^{V}\frac{|D^{v}|}{|D|} Gini(D^{v})$

或者说，假设有k个类别，样本点属于第k类的概率为$p_{k}$,则概率分布的基尼指数定义为：

$Gini(p) = \sum_{i=1}^{K} p_{i}(1-p_{i}) = 1 - \sum_{k=1}^{K} p_{k}^{2}$

### 2.1 CART决策树

CART假设决策树是一个二叉树，它通过递归地二分每个特征，将特征空间划分为有限个单元，并在这些单元上确定预测的概率分布。CART算法中，对于回归树，采用的是平方误差最小化准则；对于分类树，采用基尼指数最小化准则。


In [38]:
class decisionnode:
    def __init__(self, col=-1, value = None, results = None, tb = None, fb = None):
        self.col = col #col是待检验的判断条件所对应的列索引值
        self.value = value # value 对应于为了使结果为true， 当前列必须匹配的值
        self.results = results #保存的是针对当前分支的结果，它是一个字典
        self.tb = tb # decision node，对应于结果为true，树上相对于当前节点的子树上的节点
        self.fb = fb # desision node,对应于结果为true时，树上相对于当前节点的子树上的节点
        
        
def giniimpurity(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp =0
    for k1 in counts:
        p1 = float(counts[k1])/total
        for k2 in counts: # 这个循环是否可以用（1-p1）替换？
            if k1 == k2: continue
            p2 = float(counts[k2])/total
            imp+=p1*p2
    return imp


def giniimpurity_2(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp = 0
    for k1 in counts.keys():
        p1 = float(counts[k1])/total
        imp+= p1*(1-p1)
    return imp

# 在某一列上对数据集进行拆分，可应用于数值型或因子型变量
def divideset(rows,column, value):
    #定义一个函数，判断当前数据行属于第一组还是第二组
    split_function = None
    if isinstance(value,int) or isinstance(value,float):
        split_function = lambda row:row[column] >= value
    else:
        split_function = lambda row:row[column]==value
    # 将数据集拆分成两个集合，并返回
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return(set1,set2)

In [50]:
# 以递归方式构造树

def buildtree(rows, scoref = entropy):
    if len(rows) ==0: return decisionode()
    current_score = scoref(rows)
    
    #定义一个变量来记录最佳拆分条件
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    
    column_count = len(rows[0]) - 1
    for col in range(0, column_count):
        # 在当前列中生成一个由不同值构成的序列
        column_values = {}
        for row in rows:
            column_values[row[col]] = 1 # initialzation
        
        # try to split the dataset according to the value in this column 
        for value in column_values.keys():
            (set1,set2) = divideset(rows, col, value)
            
            # information gain
            p = float(len(set1))/len(rows)
            gain = current_score - p * scoref(set1) - (1-p)*scoref(set2)
            if gain > best_gain and len(set1) >0 and len(set2) >0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1, set2)
                
    # create sub branch
    if best_gain>0:
        trueBranch = buildtree(best_sets[0])
        falseBranch = buildtree(best_sets[1])
        return decisionnode(col = best_criteria[0], value = best_criteria[1], tb = trueBranch, fb = falseBranch)

    else:
        return decisionnode(results = uniquecounts(rows))
    

# dicision tree visualization

def printtree(tree, indent = ''):
    # if tree node or not
    if tree.results != None:
        print(str(tree.results))
    else:
        # print conditions
        print(str(tree.col) +":"+str(tree.value)+"?")
        # print indent
        print(indent+"T->")
        printtree(tree.tb, indent+"")
        print(indent+"F->")
        printtree(tree.fb,indent+" ")       
        

# classify new observations

def classify(observation,tree):
    if tree.results!= None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v,int) or isinstance(v,float):
            if v>= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v==tree.value : branch = tree.tb
            else: branch = tree.fb
        return classify(observation,branch)
        

In [51]:
my_data=[['slashdot','USA','yes',18,'None'],
        ['google','France','yes',23,'Premium'],
        ['digg','USA','yes',24,'Basic'],
        ['kiwitobes','France','yes',23,'Basic'],
        ['google','UK','no',21,'Premium'],
        ['(direct)','New Zealand','no',12,'None'],
        ['(direct)','UK','no',21,'Basic'],
        ['google','USA','no',24,'Premium'],
        ['slashdot','France','yes',19,'None'],
        ['digg','USA','no',18,'None'],
        ['google','UK','no',18,'None'],
        ['kiwitobes','UK','no',19,'None'],
        ['digg','New Zealand','yes',12,'Basic'],
        ['slashdot','UK','no',21,'None'],
        ['google','UK','yes',18,'Basic'],
        ['kiwitobes','France','yes',19,'Basic']]

In [52]:
divideset(my_data,2,'yes')

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['slashdot', 'UK', 'no', 21, 'None']])

In [53]:
giniimpurity(my_data)

0.6328125

In [54]:
giniimpurity_2(my_data)

0.6328125

In [56]:
tree = buildtree(my_data)
printtree(tree = tree)

0:google?
T->
3:21?
T->
{'Premium': 3}
F->
2:no?
 T->
{'None': 1}
 F->
{'Basic': 1}
F->
0:slashdot?
 T->
{'None': 3}
 F->
2:yes?
  T->
{'Basic': 4}
  F->
3:21?
   T->
{'Basic': 1}
   F->
{'None': 3}


In [57]:
classify(['(direct)','USA','yes',5],tree)

{'Basic': 4}

# 剪枝处理

如果对训练集建立完整的决策树，会使得模型过于针对训练数据，拟合了大部分的噪声，即出现过度拟合的现象。为了避免这个问题，有两种解决的办法：预剪枝和后剪枝。 剪枝(pruning)是决策树学习算法对付“过拟合”的主要手段。

剪枝方法和程度对决策树泛化性能的影响相当显著。

## 预剪枝“prepruning”

对每个结点在划分前先进行估计，若当前结点的划分不能带来决策树泛化能力的提升，则停止划分，当前结点标记为叶结点。

如何评估决策树泛化能力？性能评估方法，比如留出法，预留一部分数据用作“验证集”以进行性能评估。预剪枝会让很多分支都没有“展开”，降低了过拟合的风险，还减少了决策树的训练时间开销和测试时间开销。但也有“欠拟合”的风险。


## 后剪枝“post-pruning”

先从训练集生成一颗完整的决策树，然后自底向上对非叶结点进行考察，如果该结点对应的字树替换为叶结点能带来决策树泛化能力提升，把该子树替换为叶结点。

后剪枝通常比预剪枝决策树保留了更多的分支，一般，后剪枝决策树的欠拟合风险很小，泛化性能往往优于预剪枝决策树。但是后剪枝过程是生成完全决策树之后进行的，训练时间开销要大很多。


## 损失函数

决策树的剪枝通过极小化决策树整体的损失函数来实现。在提高信息增益的基础上，对模型的复杂度T施加惩罚，就是损失函数的定义：

$$C_{\alpha}(T) = \sum_{t=1}^{|T|} N_{t} H_{t}(T) + \alpha |T|$$

$\alpha$的大小反映了对模型训练集拟合度和模型复杂度的折衷考虑。剪枝的过程就是当$\alpha$确定时，选择损失函数最小的模型。

步骤如下：

1. 计算每个节点的经验熵；

2. 递归地从树的叶节点向上回缩，如果将某一个父节点的所有叶节点合并，能够使得其损失函数减小，则进行剪枝，将父节点变成新的叶节点；

3. 返回2，直到不能继续合并。

In [59]:
# 决策树的剪枝

def prune(tree,mingain):
    # 如果分支不是叶节点，则对其进行剪枝
    if tree.tb.results == None:
        prune(tree.tb,mingain)
    if tree.fb.results == None:
        prune(tree.fb,mingain)
    # 如果两个子分支都是叶节点，判断是否能够合并
    if tree.tb.results !=None and tree.fb.results !=None:
        #构造合并后的数据集
        tb,fb = [],[]
        for v,c in tree.tb.results.items():
            tb+=[[v]]*c
        for v,c in tree.fb.results.items():
            fb+=[[v]]*c
        #检查熵的减少量
        delta = entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)
        if delta < mingain:
            # 合并分支
            tree.tb,tree.fb = None,None
            tree.results = uniquecounts(tb+fb)
# test
tree = buildtree(my_data,scoref = giniimpurity)
prune(tree,0.1)
printtree(tree)

0:google?
T->
3:21?
T->
{'Premium': 3}
F->
2:no?
 T->
{'None': 1}
 F->
{'Basic': 1}
F->
0:slashdot?
 T->
{'None': 3}
 F->
2:yes?
  T->
{'Basic': 4}
  F->
3:21?
   T->
{'Basic': 1}
   F->
{'None': 3}


# 连续值和缺失值

## 连续值

到目前为止我们讨论的都是离散属性，但是现实任务中我们不可避免的会遇到连续属性，此时我们应该如何处理呢？

C4.5算法给出的方案是用二分法来对连续属性进行处理。假设样本集为D，其中有一连续属性a,假设a在D上有n个不同的取值，我们将其由小到大排序$a_{1},a_{2},...,a_{n}$。 我们设定一个划分点t将D分为两个子集$D_{t}^{−}$,$D_{t}^{+}$，前者包含属性a上小于等于t的样本，后者包含大于t的样本。显然这里t在两个相邻a值之间取任意值不会影响划分结果，那么t值就有n−1个候选项。通常我们可将划分点设为各个区间的中位点，即：

$$ T_{a} = \{ \frac{a^{i}+a^{i+1}}{2}|1\leq i \leq n-1 \}$$

那么我们就可以像离散值一样考察这些划分点，例如当使用信息增益时：

$$Gain(D,a) = max Gain(D,a,t) = max \ H(D) - \sum_{\lambda \in \{ -, + \}} \frac{|D_{t}^{\lambda}|}{|D|} H(D_{t}^{\lambda})$$

**连续属性被用来在某个节点做划分之后，仍然可以在后续节点被用到，这和离散属性不同。**

## 缺失值
现实任务中经常遇到属性值确实，比如因为测量成本、隐私保护等因素，患者的医疗数据可能在某些属性上是未知的。如果简单的放弃不完整样本，是对数据信息极大的浪费，训练出来的模型性能也会受到影响。

- 问题1： 如何在属性值缺失的情况下进行划分属性选择？

- 问题2：给定划分属性，若样本在该属性上的缺失值，如何对样本进行划分？



### 划分属性的选择

在不具有缺失值的数据集中，对于某一个属性a，算法一般使用其所有样本来判断属性的优劣。自然地，对于具有缺失值的数据集，在某一个属性a下，算法只能使用在属性a下不具有缺失值的样本子集（即下面的$\tilde{D}$）来判断属性a的优劣。那么，缺失样本所占的比例是否影响算法对属性的选择？

<mark style = background-color:yellow>对于数据集$D$,如果没有缺失值，则以信息增益最大的属性作为最优划分属性；假如样本有缺失值，其信息增益就是：无缺失值样本的比例$\rho$乘以无缺失值样本子集$\tilde{D}$的信息增益。</mark>

这说明，对于某一个属性，如果缺失值比例越大，信息增益越不重要。


假设  表示在数据集$D$的属性a上，不具有缺失值的样本构成的样本子集；$\tilde{D^{v}}$表示在$\tilde{D}$的属性a上取值为$a^{v}$时构成的样本子集；  表示在$\tilde{D_{k}}$中属于第k类的样本子集。由此，可以确定以下变量（初始时$w_{x}=1$):

$\rho = \frac{\sum_{ x\in\tilde{D}} w_{x}}{\sum_{x\in D}w_{x}}$

$\tilde{\rho_{k}} = \frac{\sum_{ x\in\tilde{D_{k}}} w_{x}}{\sum_{x\in \tilde{D}}w_{x}}$

$\tilde{r_{v}} = \frac{\sum_{ x\in\tilde{D^{v}}} w_{x}}{\sum_{x\in D}w_{x}}$


那么，具有缺失值的样本集的信息增益公式就变成了：

$$ Gain(D, a) = \rho * Gain(\tilde{D}, a) = \rho * (H(D) - \sum_{v=1}^{V} \tilde{r_{v}} H(\tilde{D^{v}}))$$


### 样本划分

因为我们不知道缺失值原本是什么，所以我们将该样本划分到所有属性分支下；但是，划分到不同分支后，该样本的权重$w_{x}$也不再相同——这个样本被划分到属性值为$a^{v}$的分支后的权重将被更新为

$$ w_{x}:= \tilde{r_{v}} w_{x}$$

即根据“先验”来决定划分到不同分支的重要程度，这个样本在下一次划分属性的选择时，将以更新后的权重值参与$\rho, \tilde{\rho_{k}},\tilde{r_{v}}$ 的再计算。同样地，对于在属性a上无缺失值的样本，被划分到子结点时的权重保持不变。

# 多变量决策树


如果把每个属性当成坐标空间中的一个坐标轴，那d个属性描述的样本就对应了d维空间中的一个数据点，对样本分类则意味着这个坐标空间中寻找到不同类样本之间的分类边界。决策树所形成的的分类边界有一个明显的特点： 轴平行(axis-parallel)，也就是它的分类边界由若干个与坐标轴平行的分段组成。

“多变量决策树”的划分不再是与坐标轴平行的线段，而是实现“斜划分”，甚至更复杂划分的决策树。以斜划分为例，此类决策树中，非叶结点不再试仅对某个属性，而是对属性的线性组合进行测试,是试图建立一个合适的线性分类器。

多变量决策树算法主要有**OC1**和Brodley and Utgoff提出的一系列算法。**OC1**先贪心地寻找每个属性的最优权值，在局部优化的基础上在对分类边界进行随机扰动以试图找到更好的边界。还有一些则引入了线性分类器学习的最小二乘法，还有一些试图在决策树的叶结点上嵌入神经网络，以结合两种学习机制的有事，如果“感知机树”在决策树的每个叶结点上训练一个感知机。

算法**ID4、ITI** 通过调整分支路径上的划分属性次序来对树进行部分重构，即在对接收到新样本后可对已学习的模型进行调整，而不用完全重新学习。

# 决策树的优缺点

## 优点

- 易于理解和解释，甚至比线性回归更直观；
- 与人类做决策思考的思维习惯契合；
- 模型可以通过树的形式进行可视化展示；
- 可以直接处理非数值型数据，不需要进行哑变量的转化，甚至可以直接处理含缺失值的数据；


## 缺点

- 对于有大量数值型输入和输出的问题，决策树未必是一个好的选择；
- 特别是当数值型变量之间存在许多错综复杂的关系，如金融数据分析；
- 决定分类的因素取决于更多变量的复杂组合时；
- 模型不够稳健，某一个节点的小小变化可能导致整个树会有很大的不同。
